In [1]:
# Read credentials

import subprocess
import os
from dotenv import load_dotenv

# Load the .env file
load_dotenv()

# Now you can access the variables
openai_api_base = os.getenv("OPENAI_API_BASE")
openai_api_key = os.getenv("OPENAI_API_KEY")


# Run the command and capture its output
completed_process = subprocess.run(source_command, shell=True, stdout=subprocess.PIPE, text=True)

# Parse the output to extract environment variables
env_output = completed_process.stdout
env_lines = env_output.splitlines()
env_variables = {}

for line in env_lines:
    key, value = line.split('=', 1)
    if any([
        "OPENAI" in key,
        "NEBULA" in key,
        "GRAPH" in key,
    ]):
        env_variables[key] = value

os.environ.update(env_variables)

In [2]:
import nltk
nltk.data.path.append('/Users/shu/Desktop/shuyherecode/llmkg/knowledge_graph/nltk_data')

In [3]:
# For OpenAI

import os

os.environ["OPENAI_API_BASE"]="https://aigptx.top/v1/"
os.environ["OPENAI_API_KEY"]=""

import logging
import sys

logging.basicConfig(
    stream=sys.stdout, level=logging.INFO
)

from llama_index import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    KnowledgeGraphIndex,
    ServiceContext,
)

from llama_index.storage.storage_context import StorageContext
from llama_index.graph_stores import NebulaGraphStore
from llama_index import set_global_service_context

import logging
import sys

from IPython.display import Markdown, display


from llama_index.llms import OpenAI


# define LLM
llm = OpenAI(temperature=0, model="gpt-3.5-turbo-16k")
service_context = ServiceContext.from_defaults(llm=llm, chunk_size=512)

# set global service context
set_global_service_context(service_context)

In [4]:
%load_ext ngql
connection_string = f"--address {os.environ['GRAPHD_HOST']} --port 9669 --user root --password {os.environ['NEBULA_PASSWORD']}"

In [5]:
%ngql {connection_string}

Connection Pool Created
INFO:nebula3.logger:Get connection to ('192.168.64.1', 9669)


,Name
0,rag_workshop
1,rag_workshop_1


In [6]:
%ngql CREATE SPACE IF NOT EXISTS rag_workshop_1(vid_type=FIXED_STRING(256), partition_num=1, replica_factor=1);

INFO:nebula3.logger:Get connection to ('192.168.64.1', 9669)


""


In [7]:
%ngql SHOW SPACES;

INFO:nebula3.logger:Get connection to ('192.168.64.1', 9669)


,Name
0,rag_workshop
1,rag_workshop_1


In [8]:
%%ngql
USE rag_workshop_1;
CREATE TAG IF NOT EXISTS entity(name string);
CREATE EDGE IF NOT EXISTS relationship(relationship string);

INFO:nebula3.logger:Get connection to ('192.168.64.1', 9669)


""


In [9]:
%ngql CREATE TAG INDEX IF NOT EXISTS entity_index ON entity(name(256));

INFO:nebula3.logger:Get connection to ('192.168.64.1', 9669)


""


In [54]:
# %ngql USE rag_workshop; CLEAR SPACE rag_workshop; # clean graph space

INFO:nebula3.logger:Get connection to ('192.168.64.1', 9669)


""


In [10]:
os.environ['NEBULA_USER'] = os.environ["NEBULA_USER"]
os.environ['NEBULA_PASSWORD'] = os.environ["NEBULA_PASSWORD"]
os.environ['NEBULA_ADDRESS'] = os.environ["NEBULA_ADDRESS"]

space_name = "rag_workshop_1"
edge_types, rel_prop_names = ["relationship"], ["relationship"]
tags = ["entity"]

graph_store = NebulaGraphStore(
    space_name=space_name,
    edge_types=edge_types,
    rel_prop_names=rel_prop_names,
    tags=tags,
)
storage_context = StorageContext.from_defaults(graph_store=graph_store)

In [11]:
## KG Building with Llama Index

from llama_index import download_loader

WikipediaReader = download_loader("WikipediaReader")

loader = WikipediaReader()

documents = loader.load_data(pages=['Guardians of the Galaxy Vol. 3'], auto_suggest=False)


In [12]:
print(documents)

[Document(id_='8e4ecab3-0a9b-48fd-a190-c188bc0908aa', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, hash='17f45b107c8d43468ffd50200b327be922190646dbd5f8f7d61f152be0d46235', text='Guardians of the Galaxy Vol. 3 (stylized in marketing as Guardians of the Galaxy Volume 3) is a 2023 American superhero film based on the Marvel Comics superhero team Guardians of the Galaxy, produced by Marvel Studios, and distributed by Walt Disney Studios Motion Pictures. It is the sequel to Guardians of the Galaxy (2014) and Guardians of the Galaxy Vol. 2 (2017), and the 32nd film in the Marvel Cinematic Universe (MCU). Written and directed by James Gunn, it features an ensemble cast including Chris Pratt, Zoe Saldaña, Dave Bautista, Karen Gillan, Pom Klementieff, Vin Diesel, Bradley Cooper, Will Poulter, Sean Gunn, Chukwudi Iwuji, Linda Cardellini, Nathan Fillion, and Sylvester Stallone. In the film, the Guardians must save Rocket\'s (Cooper

In [13]:

kg_index = KnowledgeGraphIndex.from_documents(
    documents,
    storage_context=storage_context,
    service_context=service_context,
    max_triplets_per_chunk=10,
    space_name=space_name,
    edge_types=edge_types,
    rel_prop_names=rel_prop_names,
    tags=tags,
)

INFO:httpx:HTTP Request: POST https://aigptx.top/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://aigptx.top/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://aigptx.top/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://aigptx.top/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://aigptx.top/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://aigptx.top/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://aigptx.top/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://aigptx.top/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://aigptx.top/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://aigptx.top/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://aigptx.top/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://aigpt

In [12]:
kg_index.storage_context.persist(persist_dir='./storage_graph')

NameError: name 'kg_index' is not defined

In [13]:
!ls storage_graph

docstore.json     index_store.json  vector_store.json


In [13]:
## Restore storage_context from disk
from llama_index import load_index_from_storage

storage_context = StorageContext.from_defaults(persist_dir='./storage_graph', graph_store=graph_store)
kg_index = load_index_from_storage(
    storage_context=storage_context,
    service_context=service_context,
    max_triplets_per_chunk=10,
    space_name=space_name,
    edge_types=edge_types,
    rel_prop_names=rel_prop_names,
    tags=tags,
    verbose=True,
)

INFO:llama_index.indices.loading:Loading all indices.


In [15]:
print(kg_index)

In [14]:
## Text2Cypher
from llama_index.query_engine import KnowledgeGraphQueryEngine

from llama_index.storage.storage_context import StorageContext
from llama_index.graph_stores import NebulaGraphStore

nl2kg_query_engine = KnowledgeGraphQueryEngine(
    storage_context=storage_context,
    service_context=service_context,
    llm=llm,
)


In [15]:
# activate connections
%ngql SHOW HOSTS


INFO:nebula3.logger:Get connection to ('192.168.64.1', 9669)


,Host,Port,Status,Leader count,Leader distribution,Partition distribution,Version
0,storaged0,9779,ONLINE,0,No valid partition,No valid partition,3.6.0
1,storaged1,9779,ONLINE,0,No valid partition,No valid partition,3.6.0
2,storaged2,9779,ONLINE,2,"rag_workshop:1, rag_workshop_1:1","rag_workshop:1, rag_workshop_1:1",3.6.0


In [ ]:
# r = nl2kg_query_engine.query("SHOW HOSTS")

In [16]:
question = """
Tell me about Rocket?
"""

response_nl2kg = nl2kg_query_engine.query(question)

# Cypher:

print("The Cypher Query is:")

query_string = nl2kg_query_engine.generate_query(question)

display(
    Markdown(
        f"""
```cypher
{query_string}
```
"""
    )
)

%ngql {query_string}

# Answer:

print("The Answer is:")

display(Markdown(f"<b>{response_nl2kg}</b>"))


INFO:httpx:HTTP Request: POST https://aigptx.top/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://aigptx.top/v1/chat/completions "HTTP/1.1 200 OK"
The Cypher Query is:
INFO:httpx:HTTP Request: POST https://aigptx.top/v1/chat/completions "HTTP/1.1 200 OK"



```cypher
MATCH (e1:`entity`)-[r:`relationship`]->(e2:`entity`)
WHERE e1.`entity`.`name` == 'Rocket'
RETURN e2.`entity`.`name` AS Rocket
```


INFO:nebula3.logger:Get connection to ('192.168.64.1', 9669)
The Answer is:


<b>Rocket is associated with several entities and relationships. Some of the entities related to Rocket include Adam Warlock, baby raccoon, Batch 89 test subjects, and a bounty hunter. Additionally, Rocket has relationships with friends, his past, and healing.</b>

In [19]:
question = """
who is openai's ceo?
"""

response_nl2kg = nl2kg_query_engine.query(question)

# Cypher:

print("The Cypher Query is:")

query_string = nl2kg_query_engine.generate_query(question)

display(
    Markdown(
        f"""
```cypher
{query_string}
```
"""
    )
)

%ngql {query_string}

# Answer:

print("The Answer is:")

display(Markdown(f"<b>{response_nl2kg}</b>"))

INFO:httpx:HTTP Request: POST https://aigptx.top/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://aigptx.top/v1/chat/completions "HTTP/1.1 200 OK"
The Cypher Query is:
INFO:httpx:HTTP Request: POST https://aigptx.top/v1/chat/completions "HTTP/1.1 200 OK"



```cypher
MATCH (e1:`entity`)-[:relationship]->(e2:`entity`)
WHERE e1.`name` == 'openai' AND e2.`name` == 'ceo'
RETURN e2.`name`
```


INFO:nebula3.logger:Get connection to ('192.168.64.1', 9669)
The Answer is:


<b>The CEO of OpenAI is not available in the given context information.</b>

Valina LLM + Prompts doesn't work well on all questions, fine-tuning, or few-shot ways could push further.

But Graph RAG is easier as:

The query-composing doesn't rely on the higher intelligence
Easier to enable approximate starting entities
Easier to push CoT-like task-break-down in the orchestration layer

In [20]:
kg_index_query_engine = kg_index.as_query_engine(
    retriever_mode="keyword",
    verbose=True,
    response_mode="tree_summarize",
)

In [21]:
response_graph_rag = kg_index_query_engine.query("What challenges do Rocket and Lylla face?")

display(Markdown(f"<b>{response_graph_rag}</b>"))

INFO:httpx:HTTP Request: POST https://aigptx.top/v1/chat/completions "HTTP/1.1 200 OK"
Extraced keywords: ['challenges', 'Rocket', 'Lylla']
INFO:llama_index.indices.knowledge_graph.retrievers:> Querying with idx: 44061e4b-533a-4790-b663-b0d6667169a3: The Guardians' med-packs are ineffective at healing Rocket's wounds, due to a...
INFO:llama_index.indices.knowledge_graph.retrievers:> Querying with idx: 7c2eb7cd-2e79-49fc-b7a7-25cbf3a15efc: 3 fulfills a character arc for the character writer and director James Gunn e...
INFO:llama_index.indices.knowledge_graph.retrievers:> Querying with idx: b9215a6d-83a6-492c-abfb-c548ec34ea88: Arête departs as a spaceship, with Nebula, Drax, and Mantis boarding to rescu...
INFO:llama_index.indices.knowledge_graph.retrievers:> Querying with idx: 98d4b832-8cfe-432d-896b-46be15adbf3a: === Post-production ===
In early June 2022, Daniela Melchior was revealed to ...
KG context:
The following are knowledge sequence in max depth 2 in the form of directed grap

<b>Rocket and Lylla face several challenges throughout their journey. They were both experimented on by the High Evolutionary, who sought to enhance and anthropomorphize animal lifeforms. Rocket attempted to free his friends, including Lylla, but they were ultimately killed by the High Evolutionary and his henchmen. Rocket himself was critically wounded by Adam Warlock, which posed a significant challenge to his survival. Additionally, Rocket and Lylla had to confront the emotional toll of their past experiences and the grief of losing their friends.</b>

In [22]:
response_graph_rag = kg_index_query_engine.query("Tell me about James Gunn.")

display(Markdown(f"<b>{response_graph_rag}</b>"))

INFO:httpx:HTTP Request: POST https://aigptx.top/v1/chat/completions "HTTP/1.1 200 OK"
Extraced keywords: ['James Gunn', 'James', 'Gunn']
INFO:llama_index.indices.knowledge_graph.retrievers:> Querying with idx: a603a534-8d77-4510-a506-a130f3b5081c: By mid-October, James Gunn had completed his exit settlement with Disney and ...
INFO:llama_index.indices.knowledge_graph.retrievers:> Querying with idx: 7c2eb7cd-2e79-49fc-b7a7-25cbf3a15efc: 3 fulfills a character arc for the character writer and director James Gunn e...
INFO:llama_index.indices.knowledge_graph.retrievers:> Querying with idx: affacc55-8018-4023-8f7b-1989ebfac93b: Iwuji was also announced as playing the High Evolutionary, appearing at the p...
INFO:llama_index.indices.knowledge_graph.retrievers:> Querying with idx: 9693d254-2796-45b7-8c7e-e10c57ad729f: In April 2016, Marvel Studios president Kevin Feige, the producer of the Guar...
INFO:llama_index.indices.knowledge_graph.retrievers:> Querying with idx: 9034cbdd-5713-496c-87

<b>James Gunn is a writer and director who has worked on several films, including the Guardians of the Galaxy franchise. He was initially set to write and potentially direct The Suicide Squad for Warner Bros / DC Films. However, he was later notified that he could return as the director for Guardians of the Galaxy Vol. 3. Gunn has a clear vision for the character arcs in the Guardians films and has been involved in the development of the Marvel Cosmic Universe. He has also received accolades for his work, including a Not a Number Award from People for the Ethical Treatment of Animals (PETA) for his depiction of animal testing in one of his films. Gunn has been praised for his storytelling and the emotional depth of his films.</b>

In [25]:
%ngql USE rag_workshop_1; MATCH p=(n)-[e:relationship*1..2]-() WHERE id(n) in ['James Gunn', 'James', 'Gunn'] RETURN p

INFO:nebula3.logger:Get connection to ('192.168.64.1', 9669)


,p
0,"(""James Gunn"" :entity{name: ""James Gunn""})-[:r..."
1,"(""James Gunn"" :entity{name: ""James Gunn""})-[:r..."
2,"(""James Gunn"" :entity{name: ""James Gunn""})-[:r..."
3,"(""James Gunn"" :entity{name: ""James Gunn""})-[:r..."
4,"(""James Gunn"" :entity{name: ""James Gunn""})-[:r..."
...,...
84,"(""Gunn"" :entity{name: ""Gunn""})-[:relationship@..."
85,"(""Gunn"" :entity{name: ""Gunn""})-[:relationship@..."
86,"(""Gunn"" :entity{name: ""Gunn""})-[:relationship@..."
87,"(""Gunn"" :entity{name: ""Gunn""})<-[:relationship..."


In [ ]:
%ng_draw

In [27]:
question = """
Tell me about James Gunn.
"""

response_nl2kg = nl2kg_query_engine.query(question)

# Cypher:

print("The Cypher Query is:")

query_string = nl2kg_query_engine.generate_query(question)

display(
    Markdown(
        f"""
```cypher
{query_string}
```
"""
    )
)

%ngql {query_string}

# Answer:

print("The Answer is:")

display(Markdown(f"<b>{response_nl2kg}</b>"))

INFO:httpx:HTTP Request: POST https://aigptx.top/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://aigptx.top/v1/chat/completions "HTTP/1.1 200 OK"
The Cypher Query is:
INFO:httpx:HTTP Request: POST https://aigptx.top/v1/chat/completions "HTTP/1.1 200 OK"



```cypher
MATCH (e1:`entity`)-[r:`relationship`]->(e2:`entity`)
WHERE e1.`entity`.`name` == 'James Gunn'
RETURN e2.`entity`.`name`
```


INFO:nebula3.logger:Get connection to ('192.168.64.1', 9669)
The Answer is:


<b>James Gunn is associated with various entities and relationships, including "exit settlement," "character arc," and "The Suicide Squad." Additionally, there is mention of the possibility of him returning as a director.</b>

In [28]:
%%ngql
MATCH p=(e1:`entity`)-[r:`relationship`]->(e2:`entity`)
WHERE e1.`entity`.`name` == 'James Gunn'
RETURN p

INFO:nebula3.logger:Get connection to ('192.168.64.1', 9669)


,p
0,"(""James Gunn"" :entity{name: ""James Gunn""})-[:r..."
1,"(""James Gunn"" :entity{name: ""James Gunn""})-[:r..."
2,"(""James Gunn"" :entity{name: ""James Gunn""})-[:r..."
3,"(""James Gunn"" :entity{name: ""James Gunn""})-[:r..."
4,"(""James Gunn"" :entity{name: ""James Gunn""})-[:r..."
5,"(""James Gunn"" :entity{name: ""James Gunn""})-[:r..."


In [ ]:
%ng_draw

In [49]:
from llama_index.memory import ChatMemoryBuffer

memory = ChatMemoryBuffer.from_defaults(token_limit=1500)

chat_engine = kg_index.as_chat_engine(
    chat_mode="context",
    verbose=True
)
response = chat_engine.chat("who is Rocket?")
display(Markdown(f"<b>{response}</b>"))

response = chat_engine.chat("who is Lylla?")
display(Markdown(f"<b>{response}</b>"))

# response = chat_engine.chat("who is Groot?")
# display(Markdown(f"<b>{response}</b>"))

# response = chat_engine.chat("do they all know each other?")
# display(Markdown(f"<b>{response}</b>"))

INFO:httpx:HTTP Request: POST https://aigptx.top/v1/chat/completions "HTTP/1.1 200 OK"
Extraced keywords: ['Rocket']
INFO:llama_index.indices.knowledge_graph.retrievers:> Querying with idx: b9215a6d-83a6-492c-abfb-c548ec34ea88: Arête departs as a spaceship, with Nebula, Drax, and Mantis boarding to rescu...
INFO:llama_index.indices.knowledge_graph.retrievers:> Querying with idx: 8c9a0848-de88-41ed-a40a-45f54983ec54: Guardians of the Galaxy Vol. 3 (stylized in marketing as Guardians of the Gal...
INFO:llama_index.indices.knowledge_graph.retrievers:> Querying with idx: 44061e4b-533a-4790-b663-b0d6667169a3: The Guardians' med-packs are ineffective at healing Rocket's wounds, due to a...
KG context:
The following are knowledge sequence in max depth 2 in the form of directed graph like:
`subject -[predicate]->, object, <-[predicate_next_hop]-, object_next_hop ...`
Rocket, <-[reunites with]-, Lylla, <-[revealed to be voicing]-, Linda Cardellini
Rocket, <-[accomplice of]-, Groot, <-[provided 

<b>Rocket is a fictional character in the Marvel Comics and the Marvel Cinematic Universe. He is a genetically modified raccoon and a member of the superhero team Guardians of the Galaxy. Rocket is known for his expertise in weapons and tactics, as well as his sarcastic and witty personality. He is voiced by Bradley Cooper in the MCU films.</b>

INFO:httpx:HTTP Request: POST https://aigptx.top/v1/chat/completions "HTTP/1.1 200 OK"
Extraced keywords: ['Lylla']
INFO:llama_index.indices.knowledge_graph.retrievers:> Querying with idx: b9215a6d-83a6-492c-abfb-c548ec34ea88: Arête departs as a spaceship, with Nebula, Drax, and Mantis boarding to rescu...
INFO:llama_index.indices.knowledge_graph.retrievers:> Querying with idx: 98d4b832-8cfe-432d-896b-46be15adbf3a: === Post-production ===
In early June 2022, Daniela Melchior was revealed to ...
KG context:
The following are knowledge sequence in max depth 2 in the form of directed graph like:
`subject -[predicate]->, object, <-[predicate_next_hop]-, object_next_hop ...`
Lylla, -[reunites with]->, Rocket, -[former Avenger]->, bounty hunter
Lylla, -[reunites with]->, Rocket, -[recalls]->, past
Lylla, -[reunites with]->, Rocket, -[befriended]->, Batch 89 test subjects
Lylla, -[reunites with]->, Rocket, -[attempted to free]->, friends
Lylla, <-[revealed to be voicing]-, Linda Cardellini
Ly

KeyboardInterrupt: 

In [38]:
response = chat_engine.chat("But how about Lylla?")
display(Markdown(f"<b>{response}</b>"))

INFO:httpx:HTTP Request: POST https://aigptx.top/v1/chat/completions "HTTP/1.1 200 OK"
Extraced keywords: ['Lylla']
INFO:llama_index.indices.knowledge_graph.retrievers:> Querying with idx: b9215a6d-83a6-492c-abfb-c548ec34ea88: Arête departs as a spaceship, with Nebula, Drax, and Mantis boarding to rescu...
INFO:llama_index.indices.knowledge_graph.retrievers:> Querying with idx: 98d4b832-8cfe-432d-896b-46be15adbf3a: === Post-production ===
In early June 2022, Daniela Melchior was revealed to ...
KG context:
The following are knowledge sequence in max depth 2 in the form of directed graph like:
`subject -[predicate]->, object, <-[predicate_next_hop]-, object_next_hop ...`
Lylla, -[reunites with]->, Rocket, -[former Avenger]->, bounty hunter
Lylla, -[reunites with]->, Rocket, -[recalls]->, past
Lylla, -[reunites with]->, Rocket, -[befriended]->, Batch 89 test subjects
Lylla, -[reunites with]->, Rocket, -[attempted to free]->, friends
Lylla, <-[revealed to be voicing]-, Linda Cardellini
Ly

KeyboardInterrupt: 

In [48]:
memory = ChatMemoryBuffer.from_defaults(token_limit=1500)

chat_engine = kg_index.as_chat_engine(
    chat_mode="react",
    verbose=True
)
response = chat_engine.chat("who is Rocket?")
display(Markdown(f"<b>{response}</b>"))

INFO:httpx:HTTP Request: POST https://aigptx.top/v1/chat/completions "HTTP/1.1 200 OK"
Thought: I can use the query_engine_tool to find information about Rocket.
Action: query_engine_tool
Action Input: {'input': 'Who is Rocket?'}
INFO:httpx:HTTP Request: POST https://aigptx.top/v1/chat/completions "HTTP/1.1 200 OK"
Extraced keywords: ['Rocket']
INFO:llama_index.indices.knowledge_graph.retrievers:> Querying with idx: b9215a6d-83a6-492c-abfb-c548ec34ea88: Arête departs as a spaceship, with Nebula, Drax, and Mantis boarding to rescu...
INFO:llama_index.indices.knowledge_graph.retrievers:> Querying with idx: 8c9a0848-de88-41ed-a40a-45f54983ec54: Guardians of the Galaxy Vol. 3 (stylized in marketing as Guardians of the Gal...
INFO:llama_index.indices.knowledge_graph.retrievers:> Querying with idx: 44061e4b-533a-4790-b663-b0d6667169a3: The Guardians' med-packs are ineffective at healing Rocket's wounds, due to a...
KG context:
The following are knowledge sequence in max depth 2 in the form o

<b>Rocket is a character who is a member of the Guardians of the Galaxy. He is a former Avenger and a bounty hunter. Rocket was found as a baby raccoon and was experimented on by the High Evolutionary, the leader of Orgocorp. He befriended his fellow test subjects, including the otter Lylla, the walrus Teefs, and the rabbit Floor. Rocket has a critical role in the plot, as he is critically wounded by Adam Warlock and the Guardians must find a way to save his life.</b>

In [ ]:
graph_rag_retriever_with_nl2graphquery = KnowledgeGraphRAGRetriever(
    storage_context=storage_context,
    service_context=service_context,
    llm=lc_llm,
    verbose=True,
    with_nl2graphquery=True,
)

query_engine_with_nl2graphquery = RetrieverQueryEngine.from_args(
    graph_rag_retriever_with_nl2graphquery, service_context=service_context
)

In [50]:
from llama_index.query_engine import RetrieverQueryEngine
from llama_index.retrievers import KnowledgeGraphRAGRetriever

graph_rag_retriever = KnowledgeGraphRAGRetriever(
    storage_context=storage_context,
    service_context=service_context,
    llm=llm,
    verbose=True,
)

query_engine = RetrieverQueryEngine.from_args(
    graph_rag_retriever, service_context=service_context
)

In [51]:
response = query_engine.query(
    "Who is Rocket?",
)
display(Markdown(f"<b>{response}</b>"))

INFO:httpx:HTTP Request: POST https://aigptx.top/v1/chat/completions "HTTP/1.1 200 OK"
Entities processed: ['Rocket']
INFO:httpx:HTTP Request: POST https://aigptx.top/v1/chat/completions "HTTP/1.1 200 OK"
Entities processed: ['rockets', 'rocket']
Graph RAG context:
The following are knowledge sequence in max depth 2 in the form of directed graph like:
`subject -[predicate]->, object, <-[predicate_next_hop]-, object_next_hop ...` extracted based on key entities as subject:
Rocket, <-[reunites with]-, Lylla, <-[revealed to be voicing]-, Linda Cardellini
Rocket, <-[accomplice of]-, Groot, <-[provided motion-capture for]-, Austin Freeman
Rocket, -[is critically wounded by]->, Adam Warlock, -[is created by]->, High Priestess Ayesha
Rocket, -[is critically wounded by]->, Adam Warlock, -[seeks to destroy]->, Guardians of the Galaxy
Rocket, -[is critically wounded by]->, Adam Warlock, -[is stabbed by]->, Nebula
Rocket, -[is critically wounded by]->, Adam Warlock, <-[created]-, Sovereign
Rocket

<b>Rocket is a character who has been through various experiences, including being experimented on, reuniting with Lylla, being critically wounded by Adam Warlock, and being an accomplice of Groot.</b>

In [52]:
vector_index = VectorStoreIndex.from_documents(
    documents,
    service_context=service_context
)

RetryError: RetryError[<Future at 0x7f8cea057bb0 state=finished raised APIRemovedInV1>]